# FIT (Fast Import Tool)

The FIT imports appropriate data from NCBI GEO.  This program is written in R.

The Gene Expression Omnibus (GEO) from NCBI is a public repository for genetic based experimental data. The data is mostly microarray-based results, typically measuring mRNA, genomic DNA, and protein data. At the most basic level of organization of GEO, there are three entity types that may be supplied by users: Platforms, Samples, and Series. Additionally, there is a curated entity called a GEO dataset.

A Platform record describes the list of elements on the array (e.g., cDNAs, oligonucleotide probesets, ORFs, antibodies) or the list of elements that may be detected and quantified in that experiment (e.g., SAGE tags, peptides). Each Platform record is assigned a unique and stable GEO accession number (GPLxxx). A Platform may reference many Samples that have been submitted by multiple submitters.

A Sample record describes the conditions under which an individual Sample was handled, the manipulations it underwent, and the abundance measurement of each element derived from it. Each Sample record is assigned a unique and stable GEO accession number (GSMxxx). A Sample entity must reference only one Platform and may be included in multiple Series. A Series record defines a set of related Samples considered to be part of a group, how the Samples are related, and if and how they are ordered. A Series provides a focal point and description of the experiment as a whole. Series records may also contain tables describing extracted data, summary conclusions, or analyses. Each Series record is assigned a unique and stable GEO accession number (GSExxx).

GEO DataSets (GDSxxx) are curated sets of GEO Sample data. A GDS record represents a collection of biologically and statistically comparable GEO Samples and forms the basis of GEO's suite of data display and analysis tools. Samples within a GDS refer to the same Platform, that is, they share a common set of probe elements. Value measurements for each Sample within a GDS are assumed to be calculated in an equivalent manner, that is, considerations such as background processing and normalization are consistent across the dataset. Information reflecting experimental design is provided through GDS subsets.

GeneChip RMA (GC-RMA) is an improved form of RMA that is able to use the sequence-specific probe affinities of the GeneChip probes to attain more accurate gene expression values.

**Enter the series for import:**

In [1]:
series = "GSE27562"

### Libraries used

To install libraries, from R console use:
```R
install.packages("ggplot2")
```

For Bioconductor packages first get biocLite via
```R
source("https://bioconductor.org/biocLite.R")
```
Then install packages. Example:
```R
biocLite("GEOquery")
```

In [2]:
library(GEOquery)
library(affy)
library(gcrma)
library(RColorBrewer)
library(affyPLM)
library(simpleaffy)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Welcome to Bioconductor

    

### Get/Create directories

Assumes this notebook is in notebook folder.

In [3]:
notebook_dir <- getwd()
main_dir <- dirname(notebook_dir)
gse_dir = file.path(main_dir,"gseRaw")
if (!dir.exists(gse_dir)) {
    dir.create(gse_dir)
}

### Load GEO data

Specifically we are interested in only GSE Data.

In [4]:
gse <- getGEO(GEO = series, destdir = gse_dir)
show(gse)

Found 1 file(s)
GSE27562_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)
See spec(...) for full column specifications.
File stored at: 
/Users/terek/Documents/Github/GenClass-Stability/gseRaw/GPL570.soft


$GSE27562_series_matrix.txt.gz
ExpressionSet (storageMode: lockedEnvironment)
assayData: 54675 features, 162 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM681982 GSM681983 ... GSM682143 (162 total)
  varLabels: title geo_accession ... tissue:ch1 (40 total)
  varMetadata: labelDescription
featureData
  featureNames: 1007_s_at 1053_at ... AFFX-TrpnX-M_at (54675 total)
  fvarLabels: ID GB_ACC ... Gene Ontology Molecular Function (16 total)
  fvarMetadata: Column Description labelDescription
experimentData: use 'experimentData(object)'
Annotation: GPL570 



In [5]:
suppFiles = getGEOSuppFiles(GEO = series, makeDirectory = TRUE, baseDir = gse_dir)

In [6]:
tarFiles = file.path(rownames(suppFiles)[1])
untarPath = file.path(dirname(tarFiles),"data")

In [9]:
untar(tarFiles, exdir=untarPath)
cels <- list.celfiles(untarPath, pattern = "[gz]")
cels

[1] "GSM681982.CEL.gz" "GSM681983.CEL.gz" "GSM681984.CEL.gz"
  [4] "GSM681985.CEL.gz" "GSM681986.CEL.gz" "GSM681987.CEL.gz"
  [7] "GSM681988.CEL.gz" "GSM681989.CEL.gz" "GSM681990.CEL.gz"
 [10] "GSM681991.CEL.gz" "GSM681992.CEL.gz" "GSM681993.CEL.gz"
 [13] "GSM681994.CEL.gz" "GSM681995.CEL.gz" "GSM681996.CEL.gz"
 [16] "GSM681997.CEL.gz" "GSM681998.CEL.gz" "GSM681999.CEL.gz"
 [19] "GSM682000.CEL.gz" "GSM682001.CEL.gz" "GSM682002.CEL.gz"
 [22] "GSM682003.CEL.gz" "GSM682004.CEL.gz" "GSM682005.CEL.gz"
 [25] "GSM682006.CEL.gz" "GSM682007.CEL.gz" "GSM682008.CEL.gz"
 [28] "GSM682009.CEL.gz" "GSM682010.CEL.gz" "GSM682011.CEL.gz"
 [31] "GSM682012.CEL.gz" "GSM682013.CEL.gz" "GSM682014.CEL.gz"
 [34] "GSM682015.CEL.gz" "GSM682016.CEL.gz" "GSM682017.CEL.gz"
 [37] "GSM682018.CEL.gz" "GSM682019.CEL.gz" "GSM682020.CEL.gz"
 [40] "GSM682021.CEL.gz" "GSM682022.CEL.gz" "GSM682023.CEL.gz"
 [43] "GSM682024.CEL.gz" "GSM682025.CEL.gz" "GSM682026.CEL.gz"
 [46] "GSM682027.CEL.gz" "GSM682028.CEL.gz" "GSM682029.CEL.gz"
 [49] "GSM682030.CEL.gz" "GSM682031.CEL.gz" "GSM682032.CEL.gz"
 [52] "GSM682033.CEL.gz" "GSM682034.CEL.gz" "GSM682035.CEL.gz"
 [55] "GSM682036.CEL.gz" "GSM682037.CEL.gz" "GSM682038.CEL.gz"
 [58] "GSM682039.CEL.gz" "GSM682040.CEL.gz" "GSM682041.CEL.gz"
 [61] "GSM682042.CEL.gz" "GSM682043.CEL.gz" "GSM682044.CEL.gz"
 [64] "GSM682045.CEL.gz" "GSM682046.CEL.gz" "GSM682047.CEL.gz"
 [67] "GSM682048.CEL.gz" "GSM682049.CEL.gz" "GSM682050.CEL.gz"
 [70] "GSM682051.CEL.gz" "GSM682052.CEL.gz" "GSM682053.CEL.gz"
 [73] "GSM682054.CEL.gz" "GSM682055.CEL.gz" "GSM682056.CEL.gz"
 [76] "GSM682057.CEL.gz" "GSM682058.CEL.gz" "GSM682059.CEL.gz"
 [79] "GSM682060.CEL.gz" "GSM682061.CEL.gz" "GSM682062.CEL.gz"
 [82] "GSM682063.CEL.gz" "GSM682064.CEL.gz" "GSM682065.CEL.gz"
 [85] "GSM682066.CEL.gz" "GSM682067.CEL.gz" "GSM682068.CEL.gz"
 [88] "GSM682069.CEL.gz" "GSM682070.CEL.gz" "GSM682071.CEL.gz"
 [91] "GSM682072.CEL.gz" "GSM682073.CEL.gz" "GSM682074.CEL.gz"
 [94] "GSM682075.CEL.gz" "GSM682076.CEL.gz" "GSM682077.CEL.gz"
 [97] "GSM682078.CEL.gz" "GSM682079.CEL.gz" "GSM682080.CEL.gz"
[100] "GSM682081.CEL.gz" "GSM682082.CEL.gz" "GSM682083.CEL.gz"
[103] "GSM682084.CEL.gz" "GSM682085.CEL.gz" "GSM682086.CEL.gz"
[106] "GSM682087.CEL.gz" "GSM682088.CEL.gz" "GSM682089.CEL.gz"
[109] "GSM682090.CEL.gz" "GSM682091.CEL.gz" "GSM682092.CEL.gz"
[112] "GSM682093.CEL.gz" "GSM682094.CEL.gz" "GSM682095.CEL.gz"
[115] "GSM682096.CEL.gz" "GSM682097.CEL.gz" "GSM682098.CEL.gz"
[118] "GSM682099.CEL.gz" "GSM682100.CEL.gz" "GSM682101.CEL.gz"
[121] "GSM682102.CEL.gz" "GSM682103.CEL.gz" "GSM682104.CEL.gz"
[124] "GSM682105.CEL.gz" "GSM682106.CEL.gz" "GSM682107.CEL.gz"
[127] "GSM682108.CEL.gz" "GSM682109.CEL.gz" "GSM682110.CEL.gz"
[130] "GSM682111.CEL.gz" "GSM682112.CEL.gz" "GSM682113.CEL.gz"
[133] "GSM682114.CEL.gz" "GSM682115.CEL.gz" "GSM682116.CEL.gz"
[136] "GSM682117.CEL.gz" "GSM682118.CEL.gz" "GSM682119.CEL.gz"
[139] "GSM682120.CEL.gz" "GSM682121.CEL.gz" "GSM682122.CEL.gz"
[142] "GSM682123.CEL.gz" "GSM682124.CEL.gz" "GSM682125.CEL.gz"
[145] "GSM682126.CEL.gz" "GSM682127.CEL.gz" "GSM682128.CEL.gz"
[148] "GSM682129.CEL.gz" "GSM682130.CEL.gz" "GSM682131.CEL.gz"
[151] "GSM682132.CEL.gz" "GSM682133.CEL.gz" "GSM682134.CEL.gz"
[154] "GSM682135.CEL.gz" "GSM682136.CEL.gz" "GSM682137.CEL.gz"
[157] "GSM682138.CEL.gz" "GSM682139.CEL.gz" "GSM682140.CEL.gz"
[160] "GSM682141.CEL.gz" "GSM682142.CEL.gz" "GSM682143.CEL.gz"

In [11]:
setwd(untarPath)
data = ReadAffy(filenames=cels)

In [12]:
data

Warning message:
“replacing previous import ‘AnnotationDbi::tail’ by ‘utils::tail’ when loading ‘hgu133plus2cdf’”Warning message:
“replacing previous import ‘AnnotationDbi::head’ by ‘utils::head’ when loading ‘hgu133plus2cdf’”


AffyBatch object
size of arrays=1164x1164 features (74 kb)
cdf=HG-U133_Plus_2 (54675 affyids)
number of samples=162
number of genes=54675
annotation=hgu133plus2
notes=